<a href="https://colab.research.google.com/github/kashindra-mahato/NLP/blob/main/google_scholar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q requests_html

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.4/83.4 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.8/143.8 kB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.8/106.8 kB 12.6 MB/s eta 0:00:00


In [ ]:
from time import sleep
import pandas as pd
from requests_html import HTMLSession

In [ ]:
session = HTMLSession()

In [ ]:
start_year = 2000
end_year = 2024
url = f'https://scholar.google.com/scholar?lr=lang_zh-TW|lang_en|lang_de|lang_es&q=machine+learning&hl=en&as_sdt=0,5&as_ylo={start_year}&as_yhi={end_year}'

In [ ]:
response = session.get(url)
response

<Response [200]>

In [ ]:
results = response.html.find('div.gs_r.gs_or.gs_scl')

In [ ]:
results, len(results)

([<Element 'div' class=('gs_r', 'gs_or', 'gs_scl') data-cid='pdcI9r5sCJcJ' data-did='pdcI9r5sCJcJ' data-lid='' data-aid='pdcI9r5sCJcJ' data-rp='0'>,
  <Element 'div' class=('gs_r', 'gs_or', 'gs_scl') data-cid='eucrqVB_bisJ' data-did='eucrqVB_bisJ' data-lid='' data-aid='eucrqVB_bisJ' data-rp='1'>,
  <Element 'div' class=('gs_r', 'gs_or', 'gs_scl') data-cid='3IzyYvVQwr4J' data-did='3IzyYvVQwr4J' data-lid='' data-aid='3IzyYvVQwr4J' data-rp='2'>,
  <Element 'div' class=('gs_r', 'gs_or', 'gs_scl') data-cid='IufbymTDBukJ' data-did='IufbymTDBukJ' data-lid='' data-aid='IufbymTDBukJ' data-rp='3'>,
  <Element 'div' class=('gs_r', 'gs_or', 'gs_scl') data-cid='zsRGU1-JC9sJ' data-did='zsRGU1-JC9sJ' data-lid='' data-aid='zsRGU1-JC9sJ' data-rp='4'>,
  <Element 'div' class=('gs_r', 'gs_or', 'gs_scl') data-cid='nhPaWHXTIagJ' data-did='nhPaWHXTIagJ' data-lid='' data-aid='nhPaWHXTIagJ' data-rp='5'>,
  <Element 'div' class=('gs_r', 'gs_or', 'gs_scl') data-cid='IVyR3mXITrQJ' data-did='IVyR3mXITrQJ' data-li

In [ ]:
sample = results[6]
title = sample.find('h3.gs_rt', first=True).text
authors = sample.find('div.gs_a', first=True).text
source = sample.find('h3.gs_rt a', first=True).attrs['href']
paper = sample.find('div.gs_or_ggsm a', first=True).attrs['href']
citations = sample.find('div.gs_ri div.gs_fl a')[2].text

title, authors, source, paper, citations

('[BOOK][B] Foundations of machine learning',
 'M Mohri, A Rostamizadeh, A Talwalkar - 2018 - books.google.com',
 'https://books.google.com/books?hl=en&lr=lang_zh-TW|lang_en|lang_de|lang_es&id=dWB9DwAAQBAJ&oi=fnd&pg=PR5&dq=machine+learning&ots=AznV0Rv_p3&sig=ePl2iLNX_EoECuqQfXYMwjGd4Vw',
 'https://www.academia.edu/download/86186744/lecture_3.pdf',
 'Cited by 5908')

In [ ]:
sample = results[0]
title = sample.xpath('.//h3[@class="gs_rt"]/a | .//h3[@class="gs_rt"]/span[2]', first=True).text
authors = sample.xpath('.//div[@class="gs_a"]', first=True).text
source = sample.xpath('.//h3[@class="gs_rt"]/a/@href', first=True)
paper = sample.xpath('.//div[@class="gs_or_ggsm"]/a/@href', first=True)

title, authors, source, paper, citations

('Machine learning: Trends, perspectives, and prospects',
 'MI Jordan, TM Mitchell\xa0- Science, 2015 - science.org',
 'https://www.science.org/doi/abs/10.1126/science.aaa8415',
 'http://www.cs.cmu.edu/~tom/pubs/Science-ML-2015.pdf',
 'Cited by 5908')

In [ ]:
def search_by_params(params: str) -> list:
    """Extracts and returns the results from the Google Scholar search."""
    session = HTMLSession()
    all_results = []

    for i in range(100):  # The final page number is 100 and each page has 10 results.
        response = session.get(f'https://scholar.google.com/scholar?{params}&start={i*10}')
        page_results = response.html.find('div.gs_r.gs_or.gs_scl')
        if len(page_results) < 10: break
        all_results.extend(page_results)
        sleep(.5)  # Google blocks requests if too many are sent too quickly.

    return all_results

In [ ]:
%%time
raw_results = search_by_params('q=metaverse&as_sdt=0,5&as_ylo=1989&as_yhi=2015')

CPU times: user 2.6 s, sys: 39 ms, total: 2.64 s
Wall time: 27.7 s


In [ ]:
def build_query(**kwargs) -> str:
    """Builds a Google Scholar query based the given arguments."""
    keywords = f'q={kwargs.get("keywords").replace(" ", "+")}'
    year_range = f'as_ylo={kwargs.get("start_year")}&as_yhi={kwargs.get("end_year")}'
    languages = f'lr={"|".join([f"lang_{l}" for l in kwargs.get("languages")])}'
    return f'{keywords}&{year_range}&{languages}'

In [ ]:
build_query(keywords='metaverse privacy', start_year=2000, end_year=2024, languages=['en', 'zh-TW'])

'q=metaverse+privacy&as_ylo=2000&as_yhi=2024&lr=lang_en|lang_zh-TW'

In [ ]:
response = session.get('https://scholar.google.com/citations?user=VfYhf2wAAAAJ&hl=en')
response

<Response [200]>

In [ ]:
show_more_button = response.html.xpath('.//button[@class="gs_btnPD gs_in_ib gs_btn_flat gs_btn_lrge gs_btn_lsu"]', first=True)
show_more_button

<Element 'button' type='button' id='gsc_bpf_more' class=('gs_btnPD', 'gs_in_ib', 'gs_btn_flat', 'gs_btn_lrge', 'gs_btn_lsu')>

In [ ]:
show_more_button.attrs

{'type': 'button',
 'id': 'gsc_bpf_more',
 'class': ('gs_btnPD', 'gs_in_ib', 'gs_btn_flat', 'gs_btn_lrge', 'gs_btn_lsu')}

In [ ]:
articles = response.html.xpath('.//tr[@class="gsc_a_tr"]')
articles

[<Element 'tr' class=('gsc_a_tr',)>,
 <Element 'tr' class=('gsc_a_tr',)>,
 <Element 'tr' class=('gsc_a_tr',)>,
 <Element 'tr' class=('gsc_a_tr',)>,
 <Element 'tr' class=('gsc_a_tr',)>,
 <Element 'tr' class=('gsc_a_tr',)>,
 <Element 'tr' class=('gsc_a_tr',)>,
 <Element 'tr' class=('gsc_a_tr',)>,
 <Element 'tr' class=('gsc_a_tr',)>,
 <Element 'tr' class=('gsc_a_tr',)>,
 <Element 'tr' class=('gsc_a_tr',)>,
 <Element 'tr' class=('gsc_a_tr',)>,
 <Element 'tr' class=('gsc_a_tr',)>,
 <Element 'tr' class=('gsc_a_tr',)>,
 <Element 'tr' class=('gsc_a_tr',)>,
 <Element 'tr' class=('gsc_a_tr',)>,
 <Element 'tr' class=('gsc_a_tr',)>,
 <Element 'tr' class=('gsc_a_tr',)>,
 <Element 'tr' class=('gsc_a_tr',)>,
 <Element 'tr' class=('gsc_a_tr',)>]

In [ ]:
sample = articles[0]
title = sample.xpath('.//a[@class="gsc_a_at"]/text()', first=True)
authors = sample.xpath('.//div[@class="gs_gray"][1]/text()', first=True)
journal = sample.xpath('.//div[@class="gs_gray"][2]/text()', first=True)
publish_year = sample.xpath('.//a[@class="gsc_a_ac gs_ibl"]/text()', first=True)
citations_no = sample.xpath('.//span[@class="gsc_a_h gsc_a_hc gs_ibl"]/text()', first=True)

title, authors, journal, publish_year, citations_no

('Keras', 'F Chollet', 'keras.io 10', '19951', '2015')

In [ ]:
sample = articles[4]
title = sample.xpath('.//a[@class="gsc_a_at"]/text()', first=True)
authors = sample.xpath('.//div[@class="gs_gray"][1]/text()', first=True)
journal = sample.xpath('.//div[@class="gs_gray"][2]/text()', first=True)
publish_year = sample.xpath('.//a[@class="gsc_a_ac gs_ibl"]/text()', first=True)
citations_no = sample.xpath('.//span[@class="gsc_a_h gsc_a_hc gs_ibl"]/text()', first=True)

title, authors, journal, publish_year, citations_no

('Beyond the imitation game: Quantifying and extrapolating the capabilities of language models',
 'A Srivastava, A Rastogi, A Rao, AAM Shoeb, A Abid, A Fisch, AR Brown, ...',
 'arXiv preprint arXiv:2206.04615',
 '529',
 '2022')

In [ ]:
articles = []

response = session.get('https://scholar.google.com/citations?hl=en&user=L5xeTxkAAAAJ&cstart=800&pagesize=100')
articles.extend(response.html.xpath('.//tr[@class="gsc_a_tr"]'))
print(response.html.xpath('.//tr[@class="gsc_a_tr"]'))

show_more_button = response.html.xpath('.//button[@class="gs_btnPD gs_in_ib gs_btn_flat gs_btn_lrge gs_btn_lsu"]/@disabled', first=True)
print('Is show more button disabled?', 'No' if show_more_button is None else 'Yes')

[<Element 'tr' class=('gsc_a_tr',)>, <Element 'tr' class=('gsc_a_tr',)>, <Element 'tr' class=('gsc_a_tr',)>, <Element 'tr' class=('gsc_a_tr',)>, <Element 'tr' class=('gsc_a_tr',)>, <Element 'tr' class=('gsc_a_tr',)>, <Element 'tr' class=('gsc_a_tr',)>, <Element 'tr' class=('gsc_a_tr',)>, <Element 'tr' class=('gsc_a_tr',)>, <Element 'tr' class=('gsc_a_tr',)>, <Element 'tr' class=('gsc_a_tr',)>, <Element 'tr' class=('gsc_a_tr',)>, <Element 'tr' class=('gsc_a_tr',)>, <Element 'tr' class=('gsc_a_tr',)>, <Element 'tr' class=('gsc_a_tr',)>, <Element 'tr' class=('gsc_a_tr',)>, <Element 'tr' class=('gsc_a_tr',)>, <Element 'tr' class=('gsc_a_tr',)>, <Element 'tr' class=('gsc_a_tr',)>, <Element 'tr' class=('gsc_a_tr',)>, <Element 'tr' class=('gsc_a_tr',)>, <Element 'tr' class=('gsc_a_tr',)>, <Element 'tr' class=('gsc_a_tr',)>, <Element 'tr' class=('gsc_a_tr',)>, <Element 'tr' class=('gsc_a_tr',)>, <Element 'tr' class=('gsc_a_tr',)>, <Element 'tr' class=('gsc_a_tr',)>, <Element 'tr' class=('gsc_a

In [ ]:
results = []

for article in articles:
    title = article.xpath('.//a[@class="gsc_a_at"]/text()', first=True)
    authors = article.xpath('.//div[@class="gs_gray"][1]/text()', first=True)
    journal = article.xpath('.//div[@class="gs_gray"][2]/text()', first=True)
    publish_year = article.xpath('.//a[@class="gsc_a_ac gs_ibl"]/text()', first=True)
    citations_no = article.xpath('.//span[@class="gsc_a_h gsc_a_hc gs_ibl"]/text()', first=True)
    results.append({
        'title': title,
        'authors': authors,
        'journal': journal,
        'publish_year': publish_year,
        'citations_no': citations_no,
    })

results

[{'title': 'Оn behalf of the American Heart Association Council on Epidemiology and Prevention Statistics Committee and Stroke Statistics Subcommittee',
  'authors': 'EJ Benjamin, MJ Blaha, SE Chiuve, M Cushman, SR Das',
  'journal': 'Heart disease and stroke statistics',
  'publish_year': '3',
  'citations_no': '2017'},
 {'title': 'on behalf of the American Heart Association Statistics Committee and Stroke Statistics Subcommittee, et al. AHA statistical update: executive summary: heart disease and stroke\xa0…',
  'authors': 'AS Go, D Mozaffarian, VL Roger, EJ Benjamin, JD Berry, WB Borden, ...',
  'journal': 'Circulation 129, 399-410',
  'publish_year': '3',
  'citations_no': '2014'},
 {'title': 'Reported Stroke Symptoms and their Associated Risk Factors in the Kingdom of Saudi Arabia, 2013',
  'authors': 'C El Bcheraoui, M Basulaiman, MA AlMazroa, F Daoud, M Tuffaha, ...',
  'journal': 'J Hypertens 3 (5), 1000177',
  'publish_year': '3',
  'citations_no': '2014'},
 {'title': 'Concent

In [ ]:
response = session.get('https://free-proxy-list.net')
response

<Response [200]>

In [ ]:
proxy_table = response.html.xpath('.//table[@class="table table-striped table-bordered"]', first=True)
proxy_table

<Element 'table' class=('table', 'table-striped', 'table-bordered')>

In [ ]:
ip_address = proxy_table.xpath('//td[1]/text()')
port = proxy_table.xpath('//td[2]/text()')
country = proxy_table.xpath('//td[4]/text()')
google = proxy_table.xpath('//td[6]/text()')
https = proxy_table.xpath('//td[7]/text()')

proxies = list(zip(ip_address, port, country, google, https))
proxies

[('172.245.110.29', '9080', 'Canada', 'yes', 'no'),
 ('34.122.187.196', '80', 'United States', 'no', 'no'),
 ('124.217.226.9', '3128', 'Malaysia', 'no', 'no'),
 ('49.249.155.3', '80', 'India', 'no', 'no'),
 ('200.25.254.193', '54240', 'Colombia', 'no', 'no'),
 ('128.199.252.22', '8000', 'Singapore', 'no', 'no'),
 ('1.224.3.122', '3888', 'South Korea', 'no', 'no'),
 ('45.87.68.8', '15321', 'Germany', 'no', 'no'),
 ('211.222.252.187', '80', 'South Korea', 'no', 'no'),
 ('154.236.176.28', '1981', 'Egypt', 'yes', 'no'),
 ('202.61.204.51', '80', 'Germany', 'yes', 'no'),
 ('162.223.89.84', '80', 'United States', 'yes', 'no'),
 ('155.94.241.130', '3128', 'United States', 'no', 'no'),
 ('5.252.23.206', '3128', 'Slovakia', 'no', 'no'),
 ('12.186.205.123', '80', 'United States', 'no', 'no'),
 ('45.56.84.254', '55123', 'United States', 'no', 'no'),
 ('47.243.177.210', '8088', 'Hong Kong', 'no', 'no'),
 ('115.96.208.124', '8080', 'India', 'no', 'yes'),
 ('103.9.77.25', '8888', 'Vietnam', 'no', 'no

In [ ]:
working_proxies = []

for proxy in proxies:
    ip_address = f'http://{proxy[0]}:{proxy[1]}'
    try:
        response = session.get('https://httpbin.org/ip', proxies={'http': ip_address, 'https': ip_address}, timeout=1)
        working_proxies.append(ip_address)
        print(f'{ip_address} is working!')
    except Exception as e:
        pass

http://20.210.113.32:80 is working!
http://20.24.43.214:80 is working!
http://20.206.106.192:80 is working!
http://20.205.61.143:80 is working!
http://20.111.54.16:8123 is working!


In [ ]:
from concurrent.futures import ThreadPoolExecutor

In [ ]:
def fetch_proxies() -> list:
    response = session.get('https://free-proxy-list.net')
    proxy_table = response.html.xpath('.//table[@class="table table-striped table-bordered"]', first=True)
    ip_address = proxy_table.xpath('//td[1]/text()')
    port = proxy_table.xpath('//td[2]/text()')
    country = proxy_table.xpath('//td[4]/text()')
    google = proxy_table.xpath('//td[6]/text()')
    https = proxy_table.xpath('//td[7]/text()')

    return list(zip(ip_address, port, country, google, https))

In [ ]:
def test_proxy(proxy: tuple) -> bool:
    ip_address = f'http://{proxy[0]}:{proxy[1]}'
    try:
        response = session.get('https://httpbin.org/ip', proxies={'http': ip_address, 'https': ip_address}, timeout=1)
        return ip_address
    except Exception as e:
        pass

In [ ]:
with ThreadPoolExecutor(max_workers=30) as executor:
    print(list(executor.map(test_proxy, fetch_proxies())))


[None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, Non

In [ ]:
def crawl(func):
    def wrapper(**kwargs):
        params = func(**kwargs)
        # print(params)
        # print(params['urls'])
        results = []

        for url in params['urls']:
            response = session.get(url)
            result = params['parser'](response)
            print(f'{url} results is:', result)
            print('------------------------------------------')
            results.extend(result)
            if not params['condition'](result):
                break

        return results

    return wrapper

In [ ]:
@crawl
def fetch_articles(**kwargs):
    query = build_query(**kwargs)
    urls = [f'https://scholar.google.com/scholar?hl=en&as_sdt=0,5&{query}&start={i}' for i in range(0, 1000, 10)]

    def parser(response):
        articles = response.html.xpath('.//div[@class="gs_r gs_or gs_scl"]')
        results = []

        for article in articles:
            title = article.xpath('.//h3[@class="gs_rt"]/a | .//h3[@class="gs_rt"]/span[2]', first=True).text
            authors = article.xpath('.//div[@class="gs_a"]', first=True).text
            source = article.xpath('.//h3[@class="gs_rt"]/a/@href', first=True)
            paper = article.xpath('.//div[@class="gs_or_ggsm"]/a/@href', first=True)
            citations = article.xpath('.//div[@class="gs_ri"]/div[@class="gs_fl"]/a[3]', first=True).text

            results.append({
                'title': title,
                'authors': authors,
                'source': source,
                'paper': paper,
                'citations': citations,
            })
        return results

    def condition(result):
        if len(result) != 10:
            return False
        return True

    return {
        'urls': urls,
        'parser': parser,
        'condition': condition,
    }

In [ ]:
articles = fetch_articles(keywords='history', languages=['en'])
articles

https://scholar.google.com/scholar?hl=en&as_sdt=0,5&q=history&as_ylo=None&as_yhi=None&lr=lang_en&start=0 results is: []
------------------------------------------


[]